In [1]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math
import json

In [2]:
class firefighterProblem:
    def __init__(self,fire_file, firefighter_file=None, node_file=None):
        #the section can be modified
        self.T_number = 150 #時間長度
        self.T = list([i for i in range(self.T_number + 1)]) #時間list
        self.P = {1:3,2:4} #各個消防單位時間處理的燃料量
        self.mode = "not random"
        
        self.model = gp.Model("FIREFIGHTER")
        self.M = 10
        self.epsilon=1e-4
        self.A_p = gp.tuplelist()
        self.A_f = gp.tuplelist()
        self.tau = gp.tupledict()
        self.lamb = gp.tupledict()
        self.K = set() #K=消防員集合
        self.Q = {}
        self.b = {}
        self.H = {}
        self.process = {}
        self.A_f_NEIGHBOR = {} #A_f_NEIGHBOR=與點i相鄰的點
        self.A_f_NEIGHBOR_T = {} #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點
        self.x = {}
        self.w = {}
        self.u = {}
        self.u_bar = {}
        self.v = {}
        self.v_bar = {}
        self.NODE_POS = {}
        
#        self.read_from_excel(node_file)
        self.read_from_excel(fire_file)
#         self.read_from_excel(firefighter_file)
        
    def read_from_excel(self, fileName):
            df = pd.read_excel(fileName, sheet_name = None)
            self.N_number = df["coordinates"].index[-1]+1
            self.N = set([i for i in range(1, self.N_number+1)])
            print(self.N)
            for i in df["fire_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                time = i['travel time']
                self.lamb[u,v] = time
                self.A_f.append((u,v))
            for i in self.N:
                self.A_p.append((i,i))
            for i in df["firefighter_route"].iloc:
                u = int(i['i'])
                v = int(i['j'])
                firefighterIndex = i['k']
                if firefighterIndex not in self.K:
                    self.K.add(int(firefighterIndex))
                time = i['travel time']
                self.tau[u,v,firefighterIndex] = time
                if (u,v) not in self.A_p:
                    self.A_p.append((u,v)) 
            for i in range(self.N_number):
                self.NODE_POS[i+1] = (int(df["coordinates"].iloc[i]['x']), int(df["coordinates"].iloc[i]['y']))
            if self.mode == 'random':
                for i in range(self.N_number):
                    self.Q[i+1] = random.randint(2,5)
                    self.b[i+1] = random.randint(20,30)
                    self.H[i+1] = random.randint(2,5)
            else:
                for i in range(self.N_number):
                    self.Q[i+1] = int(df["coordinates"].iloc[i]['quantity'])
                    self.b[i+1] = int(df["coordinates"].iloc[i]['value'])
                    self.H[i+1] = int(df["coordinates"].iloc[i]['burning time'])
            print(len(self.H))
            self.N_D = df['ff_source']['N_D'].tolist()
            self.P = {1: int(df['ff_source']['P'].iloc[0])}
            self.N_F = set(df['fire_source']['N_F'].tolist())
            for i in self.N_D:
                if math.isnan(i):
                    self.N_D.remove(i)
            self.N_D = set(self.N_D)
        
        #         if "fire_route" in fileName:
#             fire_df = pd.read_excel(fileName)
#             df_num = len(fire_df.index)
#             for i in range(df_num):
#                 u = int(fire_df.iloc[i]['i'])
#                 v = int(fire_df.iloc[i]['j'])
#                 time = fire_df.iloc[i]['travel time']
#                 self.lamb[u,v] = time
#                 self.A_f.append((u,v))
#         elif "firefighter_route" in fileName:
#             firefighter_df = pd.read_excel(fileName)
#             df_num = len(firefighter_df.index)
#             for i in self.N:
#                 self.A_p.append((i,i))
#             for i in range(df_num):
#                 u = int(firefighter_df.iloc[i]['i'])
#                 v = int(firefighter_df.iloc[i]['j'])
#                 firefighterIndex = firefighter_df.iloc[i]['k']
#                 if firefighterIndex not in self.K:
#                     self.K.add(int(firefighterIndex))
#                 time = firefighter_df.iloc[i]['travel time']
#                 self.tau[u,v,firefighterIndex] = time
#                 #避免在多消防員時重複紀錄arc set
#                 if (u,v) not in self.A_p:
#                     self.A_p.append((u,v))  
#         elif "nodeInformation" in fileName:
#             nodeInfo1 = pd.read_excel(fileName, 'coordinates')
#             df_num = len(nodeInfo1.index)
#             self.N_number = df_num
#             self.N = set([i for i in range(1, self.N_number+1)])
            
#             for i in range(self.N_number):
#                 self.NODE_POS[i+1] = (int(nodeInfo1.iloc[i]['x']), int(nodeInfo1.iloc[i]['y']))
#             if self.mode == 'random':
#                 for i in range(self.N_number):
#                     self.Q[i+1] = random.randint(2,5)
#                     self.b[i+1] = random.randint(20,30)
#                     self.H[i+1] = random.randint(2,5)
#             else:
#                 for i in range(self.N_number):
#                     self.Q[i+1] = int(nodeInfo1.iloc[i]['quantity'])
#                     self.b[i+1] = int(nodeInfo1.iloc[i]['value'])
#                     self.H[i+1] = int(nodeInfo1.iloc[i]['burning time'])
            
#             nodeInfo2 = pd.read_excel(fileName, 'source')
#             self.N_D = set(nodeInfo2['N_D'].tolist())
#             self.N_F = set(nodeInfo2['N_F'].tolist())
    def initialize(self):
        for k in self.K:
            self.process[k]={}
            for i in self.N:
                if i in self.N_D:
                    self.process[k][i] = 0
                else:
                    self.process[k][i] = math.ceil(self.Q[i] * self.H[i] / self.P[k])
        for l in self.N - self.N_D:                          #定義A_f_NEIGHBOR
            connect = self.A_f.select('*',l)
            self.A_f_NEIGHBOR[l]=[]
            for temp in connect:
                self.A_f_NEIGHBOR[l].append(temp[0])
        for j in self.N - self.N_D - self.N_F:                         #定義A_f_NEIGHBOR_T
            for t in self.T:
                self.A_f_NEIGHBOR_T[j, t]=[]
                for i in self.A_f_NEIGHBOR[j]:
                    if t - self.H[i] - self.lamb[i, j]>=0:
                        self.A_f_NEIGHBOR_T[j, t].append(i)
        #定義x[i,j,k,t]
        for k in self.K:
            for t in self.T:
                for i in range(len(self.A_p)):
                    self.x[self.A_p[i][0], self.A_p[i][1], k, t] = self.model.addVar(vtype='B', name="x[%d,%d,%d,%d]" % (self.A_p[i][0], self.A_p[i][1], k, t))
        #定義w[i,k,t]
        for k in self.K:
            for t in self.T:
                for i in self.N:
                    self.w[i,k,t] = self.model.addVar(vtype='B',name="w[%d,%d,%d]" % (i, k, t))

        #定義u[i,t]
        self.u = self.model.addVars(self.N, self.T, vtype="B", name="u")

        #定義u_bar[i,k,t]
        self.u_bar = self.model.addVars(self.N, self.K, self.T, vtype="B", name="u_bar")

        #定義v[i,t]
        self.v = self.model.addVars(self.N, self.T, vtype="B", name="v")
        
        #定義v_bar[i,t]
        self.v_bar = self.model.addVars(self.N, self.T, vtype="B", name="v_bar")        
        
        self.model.update()
        
        #原點flow blance
        for k in self.K:                        
            self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i,j in self.A_p) <= 1)
    
        #限定從depot出發
        for O in self.N_D:
            connect = self.A_p.select(O,'*')
            for k in self.K:
                self.model.addConstr(gp.quicksum(self.x[i,j,k,0] for i, j in connect) == 1)
                
        #depot不會被保護
        #self.model.addConstrs((self.u_bar[i,k,t]==0 for i in self.N_D for k in self.K for t in range(self.T_number+1)))

        #flow balance
        for k in self.K:
            for t in range(1,self.T_number):
                for j in self.N: 
                    in_connect = self.A_p.select('*',j)
                    out_connect = self.A_p.select(j,'*')
                    temp = 0 #in-degree
                    temp += self.w[j, k, t - 1] # t-1在j idle
                    
                    if j in self.N_D: #j in depot set會有u_bar，只是都為0
                        temp += self.u_bar[j, k, t]
                    else: #j not in depot set, 若現在的t > process time，則有u_bar且為非0
                        if self.process[k][j] <= t:
                            temp += self.u_bar[j, k, t - self.process[k][j]]
                    for m, n in in_connect:
                        if m != n and self.tau[m, n, k] <= t: #若現在的t>travel time，則會有x
                            temp += self.x[m, n, k, t - self.tau[m,n,k]]
                    self.model.addConstr(temp == gp.quicksum(self.x[n, w, k, t] for n, w in out_connect), name="flow") #in-degree = out-degree
        
        #若在t from i to i(i not include depot), 一定會是在t時刻開始保護或idle
        self.model.addConstrs((self.u_bar[i, k, t] + self.w[i, k, t] == self.x[i, i, k, t] for i in self.N - self.N_D for k in self.K for t in range(self.T_number)))
        
        #若在t from s to s(s is in deopt set), 一定會在t時刻idle
        self.model.addConstrs((self.w[s, k, t] == self.x[s, s, k, t]) for s in self.N_D for k in self.K for t in range(self.T_number))
        
        #每個node在T內只會開始燒、開始保護、或未影響
        self.model.addConstrs(self.u[i, t] + self.u_bar.sum(i, '*', t) <= 1 for i in self.N for t in self.T)
        
        #開始燒與已經燒之間的關係
        self.model.addConstrs(self.v[i, t] + self.u[i, t] == self.v[i, t+1] for i in self.N for t in self.T[0:-1])
        
        #開始保護與已經保護之間的關係
        self.model.addConstrs(self.v_bar[i, t] + self.u_bar.sum(i, '*', t) == self.v_bar[i, t+1] for i in self.N for t in self.T[0:-1]) #constrain 9
        
        #deopt在T內不會開始保護
        self.model.addConstrs(self.u_bar.sum(s, '*', '*') == 0 for s in self.N_D)
        
        #depot在T內不會開始燒
        self.model.addConstrs(self.u.sum(s, '*') == 0 for s in self.N_D)
        
        #火焰的延燒
        for j in self.N - self.N_D - self.N_F:
            for t in range(self.T_number):
                if len(self.A_f_NEIGHBOR_T[j, t]) == 0:
                    self.model.addConstr(self.u[j, t] == 0, name='test')
                else:
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) / self.M <= self.u[j, t] + self.v[j, t] + self.v_bar[j, t + 1])
                    self.model.addConstr(gp.quicksum(self.u[i, t - self.H[i] - self.lamb[i, j]] for i in self.A_f_NEIGHBOR_T[j, t]) >= self.u[j, t])
        
        #給定起火點
        for i in self.N_F:                           
            self.model.addConstr(self.u[i, 0] == 1)
        
        #給定所有節點已經燒的起始狀態
        for i in self.N:                           
            self.model.addConstr(self.v[i, 0] == 0)
    
        #給定所有節點已經保護的起始狀態
        for i in self.N:                            
            self.model.addConstr(self.v_bar[i, 0] == 0)

        #消防員不能去已經被燃燒的節點
        for k in self.K:                             
            for t in self.T:
                for l in range(len(self.A_p)):
                    if self.A_p[l][1] not in self.N_D:
                        if self.A_p[l][0] ==  self.A_p[l][1]:
                            if t + 2 <= self.T_number:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + 1]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                            else:
                                self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        elif t + self.tau[self.A_p[l][0], self.A_p[l][1], k] + 1 <= self.T_number:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], t + self.tau[self.A_p[l][0], self.A_p[l][1], k]]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
                        else:
                            self.model.addConstr(self.M * (1 - self.v[self.A_p[l][1], self.T_number]) >= self.x[self.A_p[l][0], self.A_p[l][1], k, t])
        
        #設定目標式
        self.model.setObjective(gp.quicksum(gp.quicksum(self.u[i, t] for t in self.T) * self.b[i] for i in self.N - self.N_D) + 
                           gp.quicksum(self.epsilon * self.x[i, j, k, t] for (i, j, k, t) in self.x if i != j) +
                           gp.quicksum(self.epsilon * t * self.u_bar[i, k, t] for i in self.N - self.N_D for k in self.K for t in self.T), GRB.MINIMIZE)
        return self.model
        
    def showTextSol(self):
        print("x:")
        for k in self.K:
            print()
            print("消防員%d的路徑" % k)
            temp = [elem for elem in self.x if elem[2] == k]
            for (i, j, k, t) in temp:
                if self.x[i, j, k, t].X > self.epsilon:
                    if i != j:
                        print("在時刻 %d 從node%d 移動到 node%d" % (t, i, j), " ,travel time:", self.tau[i, j, k])
                    else:            
                        if self.u_bar[i,k,t].X == 1:
                            print("在時刻 %d 對node%d進行保護" % (t,i), " ,processing time:", self.process[k][i])
                        else:
                            print("在時刻 %d 在node%d idle" % (t,i))

#         print("w:")
#         for (i, k, t) in self.w:
#             if self.w[i, k, t].X > self.epsilon:
#                 print("w[%d,%d,%d]" % (i, k, t) , self.w[i, k, t].X)

#         print("u:")
#         for (i, t) in self.u:
#             if self.u[i,t].X > self.epsilon:
#                 print("u[%d,%d]" % (i,t), self.u[i,t].X)

#         print("u_bar:")
#         for (i, k, t) in self.u_bar:
#             if self.u_bar[i, k, t].X > self.epsilon:
#                 print("u_bar[%d,%d,%d]" % (i, k, t), self.u_bar[i, k, t].X)

#         print("v:")
#         for (i, t) in self.v:
#             if self.v[i, t].X > self.epsilon:
#                 print("v[%d,%d]" % (i,t), self.v[i, t].X)

#         print("v_bar:")
#         for (i, t) in self.v_bar:
#             if self.v_bar[i, t].X > self.epsilon:
#                 print("v_bar[%d,%d]" % (i,t), self.v_bar[i, t].X)
    def writeJson(self, file):
        data = {}
        data['NODE_POS'] = self.NODE_POS
        data['N'] = list(self.N)
        data['N_D'] = list(self.N_D)
        data['N_F'] = list(self.N_F)
        data['K'] = list(self.K)
        data['A_p'] = list([str(i) for i in self.A_p])
        data['A_f'] = list([str(i) for i in self.A_f])
        data['tau'] = dict((str(i), self.tau[i]) for i in self.tau)
        data['lamb'] = dict((str(i), self.lamb[i]) for i in self.lamb)
        data['T'] = self.T
        data['q'] = self.Q
        data['b'] = self.b
        data['p'] = self.P
        data['h'] = self.H
        
        temp = {}
        for (i, j, k, t) in self.x:
            temp[str((i, j, k, t))] = self.x[i, j, k, t].X
        data['x'] = temp

        temp = {}
        for (i, k, t) in self.w:
            temp[str((i, k, t))] = self.w[i, k, t].X
        data['w'] = temp

        temp = {}
        for (i, t) in self.u:
            temp[str((i, t))] = self.u[i, t].X
        data['u'] = temp

        temp = {}
        for (i, k, t) in self.u_bar:
            temp[str((i, k, t))] = self.u_bar[i, k, t].X
        data['u_bar'] = temp

        temp = {}
        for (i, t) in self.v:
            temp[str((i, t))] = self.v[i, t].X
        data['v']  = temp

        temp = {}
        for (i, t) in self.v_bar:
            temp[str((i, t))] = self.v_bar[i, t].X
        data['v_bar'] = temp
        
        json_data = json.dumps(data)
        with open(file[:-5]+"_data.json", "w") as file:
            file.write(json.dumps(data))

In [3]:
import os
files = os.listdir("./")

file_list = [f for f in files if os.path.isfile(os.path.join("./", f))and f[-5:] == ".xlsx"]

for file in file_list:
    print(file)

    ff = firefighterProblem(fire_file=file)
    model = ff.initialize()

    #run model and write lp file
    model.optimize()
    #model.write('test.lp')
    print("optimal value : ", model.ObjVal)

    ff.showTextSol()
    ff.writeJson(file)

FFP_n20_no1.xlsx
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-26
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 31972 rows, 27482 columns and 111655 nonzeros
Model fingerprint: 0xd7f960df
Variable types: 0 continuous, 27482 integer (27482 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 25320 rows and 19200 columns
Presolve time: 0.50s
Presolved: 6652 rows, 8282 columns, 26347 nonzeros
Variable types: 0 continuous, 8282 integer (8282 binary)
Found heuristic solution: objective 155.0020000

Root relaxation: objective 6.835306e+01, 3790 iterations, 0.28 seconds (0.27 work units)

    Nodes    |    Current Node    |     Objective Bound

     0     0   91.56880    0  647  135.02560   91.56880  32.2%     -    2s
     0     0   92.42148    0  586  135.02560   92.42148  31.6%     -    2s
     0     0   92.44802    0  730  135.02560   92.44802  31.5%     -    3s
     0     0   92.47564    0  826  135.02560   92.47564  31.5%     -    3s
     0     0   92.47565    0  829  135.02560   92.47565  31.5%     -    3s
     0     0   92.76065    0  856  135.02560   92.76065  31.3%     -    3s
H    0     0                     135.0251000   92.76065  31.3%     -    3s
     0     0   92.83124    0  823  135.02510   92.83124  31.2%     -    3s
     0     0   92.83126    0  830  135.02510   92.83126  31.2%     -    3s
     0     0   92.87773    0  712  135.02510   92.87773  31.2%     -    4s
     0     0   92.90099    0  725  135.02510   92.90099  31.2%     -    4s
     0     0   92.90099    0  724  135.02510   92.90099  31.2%     -    4s
     0     0   93.10592    0  654  135.02510   93.10592  31.0%     -    4s
H    0     0             

  1285   680   95.01726   23  319  115.01830   95.01726  17.4%   146  160s
  1338   698   95.02897   36  204  115.01830   95.01726  17.4%   146  165s
  1439   614   97.97579   59  168  115.01830   95.01726  17.4%   150  170s
  1557   528     cutoff   26       115.01830   95.03509  17.4%   154  176s
H 1560   493                     115.0176000   95.03509  17.4%   154  176s
H 1560   461                     115.0170000   95.03509  17.4%   154  176s
* 1599   420              45     115.0169000  100.02669  13.0%   152  177s
  1687   400  100.03217   62  190  115.01690  100.02669  13.0%   153  181s
  1749   375  109.40884   36  143  115.01690  105.03613  8.68%   154  185s
  1875   312 infeasible   50       115.01690  105.03733  8.68%   152  190s

Cutting planes:
  Gomory: 2
  Cover: 1
  Implied bound: 2
  Clique: 6
  Zero half: 11
  RLT: 33

Explored 1900 nodes (297128 simplex iterations) in 190.19 seconds (102.53 work units)
Thread count was 8 (of 8 available processors)

Solution count 10:

     0     0   91.92251    0  691  140.05000   91.92251  34.4%     -    6s
     0     0   92.77958    0  670  140.05000   92.77958  33.8%     -    7s
     0     0   92.77958    0  669  140.05000   92.77958  33.8%     -    8s
     0     0   92.77958    0  695  140.05000   92.77958  33.8%     -    8s
     0     0   92.77966    0  697  140.05000   92.77966  33.8%     -    8s
     0     0   92.77966    0  681  140.05000   92.77966  33.8%     -   10s
     0     2   92.77966    0  681  140.05000   92.77966  33.8%     -   11s
     3     8  113.88112    2  242  140.05000   92.77966  33.8%  1985   15s
H   31    30                     140.0499000   93.67390  33.1%   471   18s
*   37    30               9     125.0393000   93.67390  25.1%   455   18s
    61    37  107.94953    6  244  125.03930  101.52778  18.8%   370   20s
H   62    37                     125.0392000  101.52778  18.8%   364   20s
H   64    37                     125.0391000  101.52778  18.8%   355   20s
H  138    99             


Solution count 10: 80.0208 80.021 80.0211 ... 130.005

Optimal solution found (tolerance 1.00e-04)
Best objective 8.002080000000e+01, best bound 8.002080000000e+01, gap 0.0000%
optimal value :  80.0208
x:

消防員1的路徑
在時刻 0 從node2 移動到 node6  ,travel time: 13.0
在時刻 13 從node6 移動到 node11  ,travel time: 10.0
在時刻 23 對node11進行保護  ,processing time: 9
在時刻 32 從node11 移動到 node3  ,travel time: 9.0
在時刻 41 從node3 移動到 node16  ,travel time: 22.0
在時刻 63 從node16 移動到 node5  ,travel time: 8.0
在時刻 71 從node5 移動到 node12  ,travel time: 6.0
在時刻 77 對node12進行保護  ,processing time: 18
在時刻 95 從node12 移動到 node5  ,travel time: 6.0
在時刻 101 對node5進行保護  ,processing time: 25
在時刻 126 在node5 idle
在時刻 127 在node5 idle
在時刻 128 在node5 idle
在時刻 129 在node5 idle
在時刻 130 在node5 idle
在時刻 131 在node5 idle
在時刻 132 在node5 idle
在時刻 133 在node5 idle
在時刻 134 在node5 idle
在時刻 135 在node5 idle
在時刻 136 在node5 idle
在時刻 137 在node5 idle
在時刻 138 在node5 idle
在時刻 139 在node5 idle
在時刻 140 在node5 idle
在時刻 141 在node5 idle
在時刻 142 在node5 idle
在時刻 143 在node5

FFP_n20_no8.xlsx
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 33050 rows, 28690 columns and 117243 nonzeros
Model fingerprint: 0x76411afc
Variable types: 0 continuous, 28690 integer (28690 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 23179 rows and 17060 columns
Presolve time: 0.80s
Presolved: 9871 rows, 11630 columns, 38776 nonzeros
Variable types: 0 continuous, 11630 integer (11630 binary)
Found heuristic solution: objective 180.0042000

Root relaxation: objective 6.850740e+01, 3784 iterations, 0.34 seconds (0.32 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [4]:
ff = firefighterProblem(fire_file="FFP_n20_no1.xlsx")
model = ff.initialize()

#run model and write lp file
model.optimize()
#model.write('test.lp')
print("optimal value : ", model.ObjVal)

ff.showTextSol()
ff.writeJson(file)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
20
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 31972 rows, 27482 columns and 111655 nonzeros
Model fingerprint: 0xd7f960df
Variable types: 0 continuous, 27482 integer (27482 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-04, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 25320 rows and 19200 columns
Presolve time: 0.57s
Presolved: 6652 rows, 8282 columns, 26347 nonzeros
Variable types: 0 continuous, 8282 integer (8282 binary)
Found heuristic solution: objective 155.0020000

Root relaxation: objective 6.835306e+01, 3790 iterations, 0.29 seconds (0.27 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0